In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

In [ ]:
%%time
adatas_pk = []
ts = [6,8,10,12,14,17,21]
for ti in ts:
    print(ti)
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/P{ti}_ATAC_L23_peakbycell_filtered.h5ad'
    adata_pk = sc.read(f)
    adata_pk.obs = adata_pk.obs.set_index('cell')
    print(adata_pk.var['peak'][:3].values)
    
    adatas_pk.append(adata_pk)
    print(adata_pk)
    break

allpeaks = adata_pk.var['peak'].values
adata_pk.var, allpeaks

In [ ]:
%%time
fout1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/qs_avc_p6to21.txt'
fout2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/l2fc_avc_p6to21.txt'

qs_avc   = np.loadtxt(fout1)
l2fc_avc = np.loadtxt(fout2)

In [ ]:
qs_avc.shape, l2fc_avc.shape

In [ ]:
l2fc_th = 1
alpha_th = 0.05
cond_sig   = np.logical_and(qs_avc < alpha_th, np.abs(l2fc_avc) > l2fc_th)
cond_sig_c = np.logical_and(qs_avc < alpha_th, l2fc_avc >  l2fc_th)
cond_sig_a = np.logical_and(qs_avc < alpha_th, l2fc_avc < -l2fc_th)

print('num AvsC-DEGs per cond:\t', cond_sig.sum(axis=1))
print('num A-DEGs per cond:\t',    cond_sig_a.sum(axis=1))
print('num C-DEGs per cond:\t',    cond_sig_c.sum(axis=1))

In [ ]:
gene_sig_instances   = cond_sig.sum(axis=0)
gene_sig_instances_a = cond_sig_a.sum(axis=0)
gene_sig_instances_c = cond_sig_c.sum(axis=0)

instances, counts = np.unique(gene_sig_instances, return_counts=True)
instances_a, counts_a = np.unique(gene_sig_instances_a, return_counts=True)
instances_c, counts_c = np.unique(gene_sig_instances_c, return_counts=True)

print('num AvsC-DEGs in num conds:\t',  counts  , np.any(cond_sig, axis=0).sum())
print('num A-DEGs in num conds:\t',     counts_a, np.any(cond_sig_a, axis=0).sum())
print('num C-DEGs in num conds:\t',     counts_c, np.any(cond_sig_c, axis=0).sum())

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.plot(np.arange(1,1+7), counts_a[1:], '-o', label='A', color='C0')
ax.plot(np.arange(1,1+7), counts_c[1:], '-o', label='C', color='C2')
ax.set_ylim(ymin=0)
ax.legend()
ax.set_ylabel('number of TFs')
ax.set_xlabel('number of time points')
sns.despine(ax=ax)
plt.show()

In [ ]:
a_any = np.array(natsorted(allpeaks[np.any(cond_sig_a, axis=0)]))
c_any = np.array(natsorted(allpeaks[np.any(cond_sig_c, axis=0)]))
a_all = np.array(natsorted(allpeaks[np.all(cond_sig_a, axis=0)]))
c_all = np.array(natsorted(allpeaks[np.all(cond_sig_c, axis=0)]))
ac_overlap = np.array(natsorted(allpeaks[np.logical_and(np.any(cond_sig_a, axis=0), np.any(cond_sig_c, axis=0))]))

print('a any', a_any.shape)
print('c any', c_any.shape)
print('a all', a_all.shape)
print('c all', c_all.shape)
print('ac overlap', ac_overlap.shape)

In [ ]:
a_all

In [ ]:
# next steps
# volcanos
# nearest genes
# tf enrichments

In [ ]:
def split_peakstring(peak):
    """
    """
    a, bc = peak.split(':')
    b, c = bc.split('-')
    b, c = int(b), int(c)
    return a,b,c

def peaksstrings_to_bed(peaks, name=None, fout=None, **kwargs):
    """
    """
    
    bedarray = np.char.replace(np.char.replace(peaks, ':', '\t'), '-', '\t')
    if name is not None:
        lastcol = np.arange(len(peaks)).astype(str)
        linker = f"\t{name}_"
        bedarray = np.char.add(np.char.add(bedarray, linker), lastcol) 
        
    if fout is not None:
        print(f"saving to {fout}")
        np.savetxt(fout, bedarray, **kwargs)
        
    return bedarray

# [(a,b,c) for a,b,c in map(split_peakstring, a_peaks[:10])]
# peaksstrings_to_bed(a_peaks, name='P6_A')


In [ ]:
import subprocess
def get_closest_genes(peaks, dout, name):
    """run bedtools using subprocess
    """
    fout_peaks = dout + f'/{name}_peaks.bed'
    fout_cgene = dout + f'/{name}_peaks_ngene.bed'
    
    # save to bed
    peaksstrings_to_bed(peaks, name=name, fout=fout_peaks, fmt='%s')

    # run bedtools and save
    # !bedtools closest -g mm10.sorted.chrom.sizes -a test.bed -b genebody.bed > out.bed
    with open(fout_cgene, 'w') as f:
        print(f"Saving to {fout_cgene}")
        result = subprocess.run(["bedtools", "closest", 
                                    "-g", "mm10.sorted.chrom.sizes", 
                                    "-t", "first",
                                    "-a", fout_peaks,
                                    "-b", "gencode.vM25.genebody_pconly.bed", #"gencode.vM25.genebody_pconly.bed",
                                   ], cwd=dout, stdout=f, stderr=subprocess.PIPE, text=True)
    print(f"Return code: {result.returncode}")

    # read the results back
    df = pd.read_csv(fout_cgene, sep='\t', header=None)
    num_uniq_genes = len(df[7].unique())
    print(f"(pks; clst 1st g, uniq g): ({len(peaks)}, {len(df)}, {num_uniq_genes})")
    
    return df

In [ ]:
dout = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac'
!mkdir -p $dout

In [ ]:
%%time
for i in range(7):
    t = ts[i]
    print(i, t)
    peaks_a = np.array(natsorted(allpeaks[cond_sig_a[i]]))
    peaks_c = np.array(natsorted(allpeaks[cond_sig_c[i]]))
    
    res_a = get_closest_genes(peaks_a, dout, f'P{t}_A') 
    res_c = get_closest_genes(peaks_c, dout, f'P{t}_C') 

# gene annot

In [ ]:
a_all_annots, a_all_styled, a_all_annots_styled = gene_modules.check_genes(a_all)
c_all_annots, c_all_styled, c_all_annots_styled = gene_modules.check_genes(c_all)

print("\t".join(a_all))
print("\t".join(a_all_annots_styled)) # _styled))

In [ ]:
print("\t".join(c_all))
print("\t".join(c_all_annots_styled)) # _styled))

In [ ]:
print("\033[0;m alltime A:", "\t".join(a_all_styled))
print("\033[0;m alltime C:", "\t".join(c_all_styled))

# Volcano

In [ ]:
def show_volcano_v2(thetypeidx, thetype, lfc, qs,
                    cond1, cond2up, cond2dn, 
                    querygenes_idx=None, 
                    gene_annots=None,
                    ax=None, bbox_to_anchor=(1,1), loc=None,
                   ): 
    """
    """
    eff = lfc[:,thetypeidx]
    pvl = -np.log10(qs[:,thetypeidx]) # +1e-10)
    cnd_up = np.all([cond1[:,thetypeidx], 
                     cond2up[:,thetypeidx]], axis=0) 
    cnd_dn = np.all([cond1[:,thetypeidx], 
                     cond2dn[:,thetypeidx]], axis=0) 

    if ax is None: 
        fig, ax = plt.subplots()
    
    # all genes
    ax.scatter(eff, pvl, s=1, color='lightgray', rasterized=True)

    # up genes
    ax.scatter(eff[cnd_up], pvl[cnd_up], s=3, facecolors='C0', rasterized=True)
    # dn genes
    ax.scatter(eff[cnd_dn], pvl[cnd_dn], s=3, facecolors='C1', rasterized=True)
    
    # # query genes
    # add text
    if querygenes_idx is not None:
        ax.scatter(eff[querygenes_idx], pvl[querygenes_idx], s=15, 
                   # label=f'type-specific (n={len(querygenes_idx):,})',
                   facecolors='none', edgecolors='k', linewidth=1, rasterized=True)
        for idx in querygenes_idx:
            ax.text(eff[idx], pvl[idx], gene_annots[idx], fontsize=10)

            
    # ax.grid(axis='y')
    sns.despine(ax=ax)
    ax.set_xlabel('log2(FC) (C/A in CP10k)')
    ax.set_ylabel('-log10(adj. p)')
    ax.set_title(f'{thetype}')
    ax.text(1,0.1,
            f'up (n={cnd_up.sum():,})\ndown (n={cnd_dn.sum():,})', 
            ha='right',
            fontsize=10, transform=ax.transAxes)
    return 

In [ ]:
unq_condidx = adata.obs['cond_order'].unique()
unq_condidx, unq_conds

In [ ]:
genes_comm = adata.var.index.values
lfc = adata.uns['l2fc_avc'].T
qs  = adata.uns['qs_avc'].T
lfc_th, qs_th = 1, 0.05

In [ ]:
cond1   = qs  <  qs_th
cond2up = lfc >  lfc_th
cond2dn = lfc < -lfc_th

In [ ]:
# typegenes_idx = basicu.get_index_from_array(genes_comm, df2.gene)

In [ ]:
thetypeidx = 0 # type A 
thetype = 'L2/3_A'
show_volcano_v2(thetypeidx, thetype, lfc, qs, cond1, cond2up, cond2dn) #  typegenes_idx)

In [ ]:
n = len(unq_conds)
fig, axs = plt.subplots(1,n,figsize=(4*n,4*1), sharex=True, sharey=True)
for cond_idx in unq_condidx:
    ax = axs.flat[cond_idx]
    thecond = unq_conds[cond_idx]
    show_volcano_v2(cond_idx, thecond, lfc, qs, cond1, cond2up, cond2dn, 
                    # typegenes_idx, 
                    ax=ax, bbox_to_anchor=(0.5, -0.3), loc='upper center')
    sns.despine(ax=ax)
fig.tight_layout()

# output = os.path.join(outfigdir, "volcano.pdf")
# powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
querygenes = ['Meis2','Foxp1','Cdh13','Cdh12']
querygenes_idx = basicu.get_index_from_array(genes_comm, querygenes) 
gene_annots = genes_comm

n = len(unq_conds)
fig, axs = plt.subplots(2,4,figsize=(4*4,4*2), sharex=True, sharey=True)
for cond_idx in unq_condidx:
    ax = axs.flat[cond_idx]
    thecond = unq_conds[cond_idx]
    show_volcano_v2(cond_idx, thecond, lfc, qs, cond1, cond2up, cond2dn, 
                    querygenes_idx=querygenes_idx, 
                    gene_annots=genes_comm,
                    ax=ax, bbox_to_anchor=(0.5, -0.3), loc='upper center')
    sns.despine(ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
# all tfs 
# not a meaningful test

In [ ]:
genes_sig = genes_comm[np.any(np.abs(lfc) > 1, axis=1)]
querygenes = np.intersect1d(annots['tf'], genes_sig) #['Meis2','Foxp1','Cdh13','Cdh12']
querygenes_idx = basicu.get_index_from_array(genes_comm, querygenes) 
gene_annots = genes_comm

n = len(unq_conds)
fig, axs = plt.subplots(2,4,figsize=(4*4,4*2), sharex=True, sharey=True)
for cond_idx in unq_condidx:
    ax = axs.flat[cond_idx]
    thecond = unq_conds[cond_idx]
    show_volcano_v2(cond_idx, thecond, lfc, qs, cond1, cond2up, cond2dn, 
                    querygenes_idx=querygenes_idx, 
                    gene_annots=genes_comm,
                    ax=ax, bbox_to_anchor=(0.5, -0.3), loc='upper center')
    sns.despine(ax=ax)
fig.tight_layout()
plt.show()